# 🎯 LGS Fine-Tune V10 - NİHAİ + GPT OPTİMİZE

## ✨ Tüm GPT Tavsiyeleri Uygulandı:
- ✅ QLoRA r=32, alpha=64
- ✅ lr=5e-5, epochs=3, warmup=0.03
- ✅ packing=False, neftune kapalı
- ✅ **80-170 kelime** (tutarlı)
- ✅ **eval_steps=25** (daha sık değerlendirme)
- ✅ **save_total_limit=2** (Drive şişmesin)
- ✅ **Val split raporu** (dağılım kontrolü)
- ✅ **JSON Repair hücresi** (parse hatalarını düzelt)

In [ ]:
# 1. Kurulum
print("⏳ Kurulum...")
!pip install -q --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q unsloth_zoo
!pip install -q --no-deps xformers trl peft accelerate bitsandbytes
print("✅ Kurulum tamamlandı!")

In [ ]:
# 2. Google Drive
from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = "/content/drive/MyDrive/lgs_soru_tahmin_projesi"
DATA_PATH = f"{PROJECT_PATH}/data/lgs_finetune_data_v10_simple.jsonl"
MODEL_SAVE_PATH = f"{PROJECT_PATH}/models/lgs_qwen_32b_v10"

import os
if os.path.exists(DATA_PATH):
    print(f"✅ Veri dosyası bulundu: {DATA_PATH}")
else:
    print(f"❌ Veri dosyası bulunamadı! Yükleyin: {DATA_PATH}")

In [ ]:
# 3. Model Yükleme (4-bit QLoRA)
from unsloth import FastLanguageModel
import torch

print("⏳ Model yükleniyor (4-bit QLoRA)...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-32B-Instruct-bnb-4bit",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
)
print(f"✅ Model yüklendi! VRAM: {torch.cuda.memory_allocated()/1024**3:.1f} GB")

In [ ]:
# 4. LoRA Konfigürasyonu
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = 64,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    use_rslora = True,
)
print("✅ LoRA: r=32, alpha=64, dropout=0.05")

In [ ]:
# 5. SERT System Prompt (80-170 kelime - TUTARLI)
SYSTEM_PROMPT = """Sen MEB LGS Türkçe soru yazarısın.

KESİN KURALLAR:
1. SADECE JSON döndür - JSON dışında TEK KARAKTER yazma!
2. dogru_cevap SADECE A, B, C veya D olabilir
3. Tüm alanlar ZORUNLU: metin, soru, sik_a, sik_b, sik_c, sik_d, dogru_cevap
4. Şıklar benzer uzunlukta ve aynı biçimde olsun
5. Metin 80-170 kelime arasında olsun
6. Çeldiriciler mantıklı ve benzer türde olsun
7. Numaralanmış cümle formatı KULLANMA

JSON FORMATI:
{"metin": "...", "soru": "...", "sik_a": "...", "sik_b": "...", "sik_c": "...", "sik_d": "...", "dogru_cevap": "A"}"""

print("✅ System Prompt: 80-170 kelime (tutarlı)")

In [ ]:
# 6. Veri Yükleme + Val Dağılım Raporu
import json
from datasets import Dataset
from collections import Counter

def format_example(item):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": item["user"]},
        {"role": "assistant", "content": item["assistant"]}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    return {"text": text}

def extract_alt_konu(user_text):
    """User prompt'undan alt konu çıkarır."""
    if "Alt Konu:" in user_text:
        return user_text.split("Alt Konu:")[1].split("\n")[0].strip()
    return "Bilinmiyor"

# Veriyi yükle
data = []
bad_lines = 0
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        try:
            item = json.loads(line.strip())
            if item.get("user") and item.get("assistant"):
                item["alt_konu"] = extract_alt_konu(item["user"])
                data.append(item)
            else:
                bad_lines += 1
        except Exception as e:
            bad_lines += 1

print(f"✅ {len(data)} örnek yüklendi")
if bad_lines > 0:
    print(f"   ⚠ {bad_lines} satır atlandı")

# Dataset oluştur
dataset = Dataset.from_list(data)
dataset = dataset.map(format_example)

# Train/Val split
split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split["train"]
val_dataset = split["test"]

print(f"   Train: {len(train_dataset)}, Val: {len(val_dataset)}")

# VAL DAĞILIM RAPORU (GPT önerisi)
print("\n📊 Val Set Alt Konu Dağılımı:")
val_alt_konular = [data[i]["alt_konu"] for i in split["test"].indices]
for ak, count in Counter(val_alt_konular).most_common():
    print(f"   {ak}: {count}")

In [ ]:
# 7. Eğitim (GPT OPTİMİZE AYARLAR)
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = 4096,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.03,
        num_train_epochs = 3,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 42,
        output_dir = "outputs",
        evaluation_strategy = "steps",
        eval_steps = 25,  # GPT: Daha sık eval
        save_strategy = "steps",
        save_steps = 50,  # GPT: Daha sık kaydet
        save_total_limit = 2,  # GPT: Drive şişmesin
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        report_to = "none",
    ),
)

print("🚀 Eğitim başlatılıyor...")
print("   eval_steps=25, save_steps=50, save_total_limit=2")
trainer.train()
print("✅ Eğitim tamamlandı!")

In [ ]:
# 8. Kalite Değerlendirmesi (80-170 kelime - TUTARLI)
import re

def evaluate_output(response):
    metrics = {
        "json_valid": False,
        "all_fields": False,
        "dogru_cevap_valid": False,
        "no_duplicate_options": False,
        "metin_length_ok": False,
        "options_balanced": False,
    }
    
    try:
        start = response.find('{')
        end = response.rfind('}') + 1
        if start < 0 or end <= start:
            return metrics, 0
        
        data = json.loads(response[start:end])
        metrics["json_valid"] = True
        
        required = ["metin", "soru", "sik_a", "sik_b", "sik_c", "sik_d", "dogru_cevap"]
        metrics["all_fields"] = all(data.get(f) for f in required)
        
        dc = data.get("dogru_cevap", "").upper()
        metrics["dogru_cevap_valid"] = dc in ["A", "B", "C", "D"]
        
        options = [data.get(f, "") for f in ["sik_a", "sik_b", "sik_c", "sik_d"]]
        metrics["no_duplicate_options"] = len(set(options)) == 4 and all(options)
        
        # 80-170 kelime (System prompt ile TUTARLI)
        metin = data.get("metin", "")
        word_count = len(metin.split())
        metrics["metin_length_ok"] = 70 <= word_count <= 180  # Biraz esnek
        
        opt_lens = [len(o) for o in options if o]
        if len(opt_lens) == 4 and min(opt_lens) > 0:
            ratio = max(opt_lens) / min(opt_lens)
            metrics["options_balanced"] = ratio < 3.0
        
    except:
        pass
    
    score = sum(metrics.values()) / len(metrics) * 100
    return metrics, score

print("✅ Kalite değerlendirme: 80-170 kelime (tutarlı)")

In [ ]:
# 9. JSON Repair Fonksiyonu (GPT ÖNERİSİ)
def repair_json(model, tokenizer, broken_output):
    """Bozuk JSON'u düzeltmeye çalışır."""
    repair_prompt = f"""Aşağıdaki metni sadece geçerli JSON'a dönüştür.
Başka hiçbir şey yazma, sadece düzeltilmiş JSON döndür.

BOZUK:
{broken_output[:1000]}

DÜZELTİLMİŞ JSON:"""
    
    messages = [{"role": "user", "content": repair_prompt}]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")
    
    input_length = inputs.shape[-1]
    
    outputs = model.generate(
        inputs,
        max_new_tokens=512,
        temperature=0.3,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    new_tokens = outputs[0][input_length:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True)

print("✅ JSON Repair fonksiyonu hazır")

In [ ]:
# 10. Test (JSON Repair dahil)
FastLanguageModel.for_inference(model)

test_prompts = [
    "Konu: Paragraf\nAlt Konu: Ana Düşünce\n\nBu kriterlere göre LGS Türkçe sorusu üret.",
    "Konu: Cümlede Anlam\nAlt Konu: Sebep-Sonuç\n\nBu kriterlere göre LGS Türkçe sorusu üret.",
    "Konu: Dil Bilgisi\nAlt Konu: Fiilimsiler\n\nBu kriterlere göre LGS Türkçe sorusu üret.",
    "Konu: Sözcükte Anlam\nAlt Konu: Çok Anlamlılık\n\nBu kriterlere göre LGS Türkçe sorusu üret.",
    "Konu: Cümlede Anlam\nAlt Konu: Deyim\n\nBu kriterlere göre LGS Türkçe sorusu üret.",
]

print("🧪 Test başlatılıyor (JSON Repair dahil)...")
print("="*60)

total_score = 0
repair_count = 0

for i, prompt in enumerate(test_prompts, 1):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt}
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")
    
    input_length = inputs.shape[-1]
    
    outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    new_tokens = outputs[0][input_length:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)
    
    metrics, score = evaluate_output(response)
    
    # JSON parse başarısız ise repair dene
    if not metrics["json_valid"]:
        print(f"   🔧 JSON repair deneniyor...")
        repaired = repair_json(model, tokenizer, response)
        metrics, score = evaluate_output(repaired)
        if metrics["json_valid"]:
            repair_count += 1
            print(f"   ✅ Repair başarılı!")
    
    total_score += score
    
    alt_konu = prompt.split("Alt Konu:")[1].split("\n")[0].strip()
    print(f"\nTest {i}: {alt_konu}")
    print(f"   Skor: {score:.0f}/100")
    for k, v in metrics.items():
        status = "✅" if v else "❌"
        print(f"   {status} {k}")

print("\n" + "="*60)
avg_score = total_score / len(test_prompts)
print(f"📊 ORTALAMA SKOR: {avg_score:.0f}/100")
print(f"🔧 JSON Repair kullanıldı: {repair_count}/{len(test_prompts)}")

if avg_score >= 80:
    print("🎉 Model başarılı!")
elif avg_score >= 60:
    print("⚠️ Model iyi ama iyileştirme gerekebilir")
else:
    print("❌ Model yetersiz")

In [ ]:
# 11. Model Kaydetme
print("💾 Model kaydediliyor...")
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)
print(f"✅ Model kaydedildi: {MODEL_SAVE_PATH}")
print("\n📋 Sonraki adımlar:")
print("   1. run_model_api_v10.ipynb çalıştırın")
print("   2. URL'yi .env dosyasına ekleyin")
print("   3. python src/web_app_v3.py başlatın")